In [ ]:
!pip install bing-image-downloader

In [2]:
!mkdir images

In [ ]:
from bing_image_downloader import downloader
downloader.download("tiger",limit=30,output_dir='images',adult_filter_off=True)

In [ ]:
from bing_image_downloader import downloader
downloader.download("panda",limit=30,output_dir='images',adult_filter_off=True)

In [ ]:
from bing_image_downloader import downloader
downloader.download("giraffe",limit=30,output_dir='images',adult_filter_off=True)

In [6]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage.io import imread 
from skimage.transform import resize

target = []
images = []
flat_data = []

DATADIR = '/content/images'
CATEGORIES = ['tiger','panda','giraffe']

for category in CATEGORIES:
  class_num = CATEGORIES.index(category)
  path = os.path.join(DATADIR,category)
  for img in os.listdir(path):
    img_array = imread(os.path.join(path,img))
    img_resized = resize(img_array,(150,150,3))
    flat_data.append(img_resized.flatten())
    images.append(img_resized)
    target.append(class_num)


flat_data = np.array(flat_data)
target = np.array(target)
images = np.array(images)



In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(flat_data,target,test_size=0.3,random_state=32)


In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm 
param_grid = [
                 {'C':[1,10,100,1000],'kernel':['linear']},
                 {'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']},
]

svc = svm.SVC(probability=True)
clf = GridSearchCV(svc,param_grid)
clf.fit(x_train,y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [9]:
y_pred = clf.predict(x_test)
y_pred

array([2, 0, 2, 0, 1, 1, 0, 1, 2, 0, 0, 2, 0, 2, 0, 0, 1, 2, 0, 2, 2, 1,
       2, 0, 0, 2, 2])

In [10]:
y_test

array([1, 0, 2, 1, 1, 1, 0, 0, 2, 2, 0, 0, 2, 0, 1, 2, 1, 2, 0, 0, 2, 0,
       2, 0, 1, 1, 2])

In [11]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [12]:
accuracy_score(y_pred,y_test)

0.5185185185185185

In [13]:
confusion_matrix(y_pred,y_test)

array([[5, 3, 3],
       [2, 3, 0],
       [3, 2, 6]])

In [14]:
import pickle
pickle.dump(clf,open('img_model.p','wb'))

In [15]:
model = pickle.load(open('img_model.p','rb'))

In [ ]:
flat_data = []
url = input("Enter URL")
img = imread(url)
img_resized = resize(img,(150,150,3))
flat_data.append(img_resized.flatten())
flat_data = np.array(flat_data)
print(img.shape)
plt.imshow(img_resized)
y_out  = model.predict(flat_data)
y_out = CATEGORIES[y_out[0]]
print(f'PREDICTED OUTPUT: {y_out}')

In [ ]:
!pip install streamlit
!pip install pyngrok==4.1.1
from pyngrok import ngrok

In [18]:
%%writefile app.py
import streamlit as st
import numpy as np
from skimage.io import imread 
from skimage.transform import resize
import pickle
from PIL import Image
st.title('Image Classifier using machine learning')
st.text('Upload the image')
model = pickle.load(open('img_model.p','rb'))
uploaded_file = st.file_uploader("Choose an image...",type="jpg")
if uploaded_file is not None:
  img = Image.open(uploaded_file)
  st.image(img,caption='Uploaded Image')

  if st.button('PREDICT'):
    CATEGORIES = ['tiger','panda','giraffe']
    st.write('Result...')
    flat_data = []
    img = np.array(img)
    img_resized = resize(img,(150,150,3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)
    y_out = model.predict(flat_data)
    y_out = CATEGORIES[y_out[0]]
    st.write(f'PREDICTED OUTPUT: {y_out}')


Writing app.py


In [19]:
!nohup streamlit run app.py &

url = ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://25b0c12bc097.ngrok.io'